# 1) Test split() function

In [2]:
import numpy as np
import matplotlib.pyplot as plt

show = False # Turn it to True if you want to plot input data in a image
# Input data --------------------------------------------
i_matrix = np.array([[0, 1, 0, 0], [0, 0, 1, 0]])
print('Original matrix: \n', i_matrix)
if show:
    plt.matshow(i_matrix)
    plt.title('Original matrix')
    plt.show()

# Function ----------------------------------------------
def split(matrix, depth):
    '''
    matrix: matrix shape(mxn)
    Depth: a number > 1
    '''
    n_tensor = [] # new created tensor
    for i in range(0, depth):
        # Creating the padding zero to compensate for the lost path of the shifted matrix
        r_matrix = len(matrix)
        pad = np.zeros((r_matrix, i))

        # 1: a new matrix is generated by concatenate the cut matrix and zero padding
        # 2: Collecting new matrices into a tensor
        n_tensor.append(np.concatenate((matrix[:, i:], pad), axis=1))
    return np.array(n_tensor)

# Show each lice of tensor after using split() function
print('\n' + '#'*15 + ' Print each slice of tensor after using split function ' + '#'*10)
tensor = split(i_matrix, 4)
for idx, i in enumerate(tensor):
    print(f'\nDepth number {idx+1}:')
    print(i)

Original matrix: 
 [[0 1 0 0]
 [0 0 1 0]]

############### Print each slice of tensor after using split function ##########

Depth number 1:
[[0. 1. 0. 0.]
 [0. 0. 1. 0.]]

Depth number 2:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]]

Depth number 3:
[[0. 0. 0. 0.]
 [1. 0. 0. 0.]]

Depth number 4:
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]]


# 2) Test yield

In [5]:
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt

def audio_source(name, window):
    sample_rate, samples = wavfile.read(name)
    begin = 0

    while begin < len(samples):
        end = min(begin + window, len(samples))
        yield samples[begin:end]
        begin = end

    print('len(samples) == end?', len(samples) == end)

# here len(piece) is just for showing each piece
generator = [len(piece) for piece in audio_source('wav/apple_and_lemmon.wav', 7000)]
print(list(generator))

len(samples) == end? True
[7000, 7000, 7000, 7000, 1376]


# 3) Test receiver() function

In [1]:
import numpy as np
from scipy.io import wavfile


def audio_source(name, window):
    sample_rate, samples = wavfile.read(name)
    begin = 0

    while begin < len(samples):
        end = min(begin + window, len(samples))
        yield samples[begin:end]
        begin = end

    print('len(samples) == end?', len(samples) == end)

def min_(data, s=False):
    yield np.min(data)

def max_(data, i=1):
    yield np.max(data)

def length_(data):
    yield len(data)

for data in audio_source('wav/apple_and_lemmon.wav', 7000):
    mi = next(min_(data))
    ma = next(max_(data))
    le = next(length_(data))
    print(f'\nMin: {mi}, max: {ma}, length {le}')


Min: -15045, max: 15733, length 7000

Min: -8768, max: 7947, length 7000

Min: -6330, max: 11097, length 7000

Min: -5881, max: 10405, length 7000

Min: -40, max: 1, length 1376
len(samples) == end? True


# 4) Test ma(), stair() and stra() while wav_to_wavelet() using yield

In [7]:
from utils.tools import stairway, stra, wavelet_to_moving_average
from os.path import join
from os import listdir
import numpy as np
from scipy.io import wavfile
from ssqueezepy import ssq_cwt
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.neighbors import NearestCentroid

# Get wavelet from .wav file
def wav_to_wavelet(path):
    '''
    path: Direction to .wav file
    '''
    sample_rate, x = wavfile.read(path) 
    twx, wx, *_ = ssq_cwt(x)  # use wx
    yield np.abs(wx)

path = 'wav/'
window = 1000 # pooling window in MA function
bins = 10 # number of bins in stairway function
low_row = 150 # get lower limit row
up_row = 155 # get upper limit row
dis = 1000 # distance between columns in stra function

for name in listdir(path):
    dir = join(path, name)
    ma_hist = stairway(wavelet_to_moving_average(next(wav_to_wavelet(dir)), window), bins)
    print('Shape of stair wave: ', ma_hist.shape)
    ma_hist_stra = stra(ma_hist, dis = dis)
    print('Shape of straight wave', ma_hist_stra.shape)

Shape of stair wave:  (277, 28377)
Shape of straight wave (277, 28377)
